# UPA
Jakub Zárybnický (xzaryb00), Matěj Mlejnek (xmlejn04) - třetí člen se během semestr rozhodl s námi už dále nekomunikovat, pokračovali jsme ve dvou.

Pro správné fungování notebooku je potřeba mít v Jupyteru mít povolenou/nainstalovanou [integraci s Matplotlib](https://github.com/matplotlib/ipympl) a v prostředí Python kernelu mít nainstalované balíčky:
- psycopg2
- pymongo
- pandas
- matplotlib

In [1]:
%matplotlib widget
import sys; sys.path.insert(0, '.')

import csv
from datetime import date, datetime
import json
import os
import time

from bson.json_util import dumps
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
from psycopg2 import extensions
import pandas as pd
import pandas.io.sql as sqlio
import requests

from db_connects import MONGO_DB_CURRENCIES, MONGO_DB_COL_CURRENCIES, connect_to_postgres, connect_to_mongodb
from scrape import parse

Některé součásti řešení zde nebudeme demonstrovat v celé délce, použijeme funkce pro zpracování vstupních dat nebo pro připojení k databázím, které máme předdefinované v našich knihovních souborech. Hlavní strukturu zde ale nastíníme, počínaje stáhnutím a zpracováním vstupních souborů.

## Stažení zdrojových souborů

In [2]:
scrape_dir = 'scraped/'
if not os.path.isdir(scrape_dir):
    os.mkdir(scrape_dir)
len(os.listdir(scrape_dir))

0

In [3]:
def scrape(base_url, output_dir, start_date, end_date):
    any_downloads = False
    for ordinal in range(start_date.toordinal(), end_date.toordinal()):
        url = base_url + date.fromordinal(ordinal).strftime('%d.%m.%Y')
        path = date.fromordinal(ordinal).strftime('%Y-%m-%d') + '.txt'
        filename = os.path.join(output_dir, path)
        if os.path.isfile(filename):
            continue
        any_downloads = True
        print("Requesting %s..." % url, end='')
        try:
            request = requests.get(url)
            if not request.text:
                print(' Empty!')
                continue
            print(' OK')
            with open(filename, 'w') as handle:
                handle.write(request.text)
            time.sleep(0.2)
        except Exception as ex:
            print(' %s' % ex)
            continue
    if not any_downloads:
        print("All files already present.")

start_date = datetime.today() - relativedelta(months=4)
end_date = datetime.today()
scrape(
    base_url='https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=',
    start_date=start_date,
    end_date=end_date,
    output_dir=scrape_dir,
)
print("Got %s input files" % len(os.listdir(scrape_dir)))
print()
with open(scrape_dir + '/' + os.listdir(scrape_dir)[0], 'r') as f:
    print(f.read())

Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=08.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=09.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=10.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=11.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=12.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=13.08.2020... OK
Requesting https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/denni_kurz.txt?date=14.08.2020... OK
Requesting ht

In [4]:
for input_file in parse(scrape_dir):
    print(input_file)
    break

{'date': datetime.datetime(2020, 10, 9, 0, 0), 'currency': {'country': 'Austrálie', 'name': 'dolar', 'code': 'AUD'}, 'lotSize': '1', 'price': '16,526'}


Nyní máme stažené všechny textové/CSV vstupní soubory a zpracované v takovém formátu, že je můžeme přímo vložit do MongoDB bez dalšího zpracování. Do tohoto bodu se veškeré zpracování skládalo z načtení CSV souboru a přidání data ke každému řádku tak, se dá dále zpracovávat.

In [5]:
client = connect_to_mongodb()
collection = client[MONGO_DB_CURRENCIES][MONGO_DB_COL_CURRENCIES]
collection.drop()

res = collection.insert_many(parse(scrape_dir))
print("Loaded %s records to MongoDB" % len(res.inserted_ids))

collection.find_one()

Loaded 4026 records to MongoDB


{'_id': ObjectId('5fcf45bf88a5bbd6ef3bbaf7'),
 'date': datetime.datetime(2020, 10, 9, 0, 0),
 'currency': {'country': 'Austrálie', 'name': 'dolar', 'code': 'AUD'},
 'lotSize': '1',
 'price': '16,526'}

Takto vypadají všechny záznamy v MongoDB. Nyní je převedeme to PostgreSQL, konkrétně do normalizovaného formátu ve dvou tabulkách, jedna tabulka měn a jedna tabulka kurzů.

- `Měna = Kód měny (string, primární klíč) x Název (string) x Země (string)`
- `Kurz = Den (date) x Kód měny (cizí klíč) x Normalizovaný kurz (Float)`

(Float sice není ideální reprezentace pro finanční výpočty, ale pro naše účely postačuje.)

In [6]:
conn = connect_to_postgres()

conn.set_isolation_level(extensions.ISOLATION_LEVEL_AUTOCOMMIT)

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS kurz")
cursor.execute("DROP TABLE IF EXISTS mena")
cursor.execute("CREATE TABLE mena (zeme varchar(100), nazev varchar(100), kod varchar(10) primary KEY)")
cursor.execute(
    "CREATE TABLE kurz (den DATE, kod varchar(10), "
    "CONSTRAINT fk_mena FOREIGN KEY(kod) REFERENCES mena(kod) ON DELETE SET NULL, "
    "normalizovany_kurz FLOAT)"
)

In [7]:
mena_res = collection.find({}, {"currency": 1, "_id": 0}).distinct("currency")
for mena_item in mena_res:
    cursor.execute("INSERT INTO mena VALUES ('{}', '{}', '{}')".format(
        mena_item["country"],
        mena_item["name"],
        mena_item["code"]
    ))

for item in collection.find({}, {"_id": 0}):
    cursor.execute("INSERT INTO kurz VALUES ('{}', '{}', '{}')".format(
        item["date"].strftime("%Y-%m-%d"),
        item["currency"]["code"],
        float(item["price"].replace(',', '.')) / int(item["lotSize"])
    ))

In [8]:
cursor.execute("SELECT * from mena")
print("%s rows" % cursor.rowcount)
for row in cursor:
    print(row)
    break
print()
cursor.execute("SELECT * from kurz")
print("%s rows" % cursor.rowcount)
for row in cursor:
    print(row)
    break

33 rows
('Austrálie', 'dolar', 'AUD')

4026 rows
(datetime.date(2020, 10, 9), 'AUD', 16.526)


Nyní máme všechna data ve strukturované reprezentaci v PostgreSQL a můžeme se pustit do jednotlivých úkolů.

## Úkol A

První úkol, který jsme si ze zadání vybrali, je vytvoření žebříčku měn, které v daném období nejvíce posílily/oslabily.

In [9]:
cursor.execute(
    "select kod, normalizovany_kurz from kurz where den = (SELECT MIN(den) from kurz)"
    " ORDER BY kod ASC"
)
min_hash = dict(cursor)
cursor.execute(
    "select kod, normalizovany_kurz from kurz where den = (SELECT MAX(den) from kurz)"
    " GROUP BY kod, normalizovany_kurz ORDER BY kod ASC"
)
diff = {}
for item in cursor:
    diff[item[0]] = min_hash[item[0]] - item[1]
diff = {k: v for k, v in sorted(diff.items(), key=lambda x: -x[1])}

fig = plt.figure()
x = np.arange(len(diff))
plt.bar(x, height=diff.values())
plt.xticks(x, diff.keys(), rotation=-90);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
print("Between %s and %s the best performing currency was %s which changed by %s units." % (
    start_date.date(), end_date.date(), list(diff.items())[0][0], round(list(diff.items())[0][1], 2)
))

Between 2020-08-08 and 2020-12-08 the best performing currency was USD which changed by 0.41 units.


## Úkol 2

Druhý úkol je nalezení skupin měn s podobným chováním (skupiny měn, které obvykle současně posilují/oslabují) pomocí korelační matice.

In [11]:
sql = "SELECT * FROM kurz ORDER BY den ASC"
df = sqlio.read_sql_query(sql, conn, parse_dates="den")
df = df.pivot_table(columns='kod', index="den", values="normalizovany_kurz")
df

kod,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,HKD,...,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR
den,,,,,,,,,,,,,,,,,,,,,
2020-08-07,16.020,13.436,4.155,16.669,24.326,3.196,3.528,26.280,29.080,2.870,...,5.962,5.432,0.30212,2.547,16.220,0.71388,3.091,22.241,31.423,1.269
2020-08-10,15.924,13.388,4.112,16.637,24.269,3.196,3.516,26.185,29.046,2.872,...,5.948,5.415,0.30235,2.547,16.207,0.71482,3.043,22.262,31.411,1.256
2020-08-11,15.941,13.371,4.085,16.706,24.344,3.197,3.512,26.155,29.110,2.864,...,5.942,5.409,0.30522,2.543,16.188,0.71480,3.062,22.194,31.223,1.268
2020-08-12,15.815,13.352,4.095,16.692,24.280,3.194,3.507,26.115,28.866,2.862,...,5.931,5.401,0.30258,2.548,16.153,0.71302,3.034,22.185,31.273,1.273
2020-08-13,15.814,13.360,4.101,16.708,24.232,3.179,3.508,26.125,28.908,2.849,...,5.942,5.403,0.29974,2.543,16.098,0.71119,3.010,22.082,31.090,1.266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-01,16.121,13.416,4.128,16.900,24.216,3.336,3.525,26.240,29.213,2.828,...,5.860,5.387,0.28906,2.569,16.348,0.72461,2.787,21.922,31.351,1.431
2020-12-02,16.121,13.501,4.195,16.909,24.406,3.335,3.548,26.410,29.177,2.823,...,5.904,5.420,0.28919,2.566,16.323,0.72402,2.793,21.887,31.277,1.423
2020-12-03,16.150,13.509,4.193,16.846,24.394,3.322,3.549,26.420,29.246,2.806,...,5.906,5.422,0.29087,2.564,16.284,0.72074,2.783,21.748,31.143,1.425


In [12]:
corr = df.corr()
corr

kod,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,HKD,...,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR
kod,,,,,,,,,,,,,,,,,,,,,
AUD,1.000000,0.829481,0.462896,0.859692,0.777986,0.795751,0.833132,0.830476,0.704479,0.761464,...,0.501763,0.792035,0.041825,0.716069,0.854546,0.748644,-0.159532,0.762278,0.817432,0.578227
BGN,0.829481,1.000000,0.210397,0.951029,0.984548,0.831253,0.999586,0.999878,0.818435,0.940854,...,0.528102,0.987958,0.208044,0.801641,0.953759,0.783112,-0.113859,0.940707,0.966517,0.429337
BRL,0.462896,0.210397,1.000000,0.206443,0.133959,0.237993,0.219477,0.213423,-0.051492,0.100125,...,0.248805,0.172912,0.192146,0.217256,0.193393,0.238843,0.096761,0.100036,0.168784,0.282855
CAD,0.859692,0.951029,0.206443,1.000000,0.940351,0.905083,0.950745,0.951474,0.884265,0.902996,...,0.403542,0.919373,0.047281,0.866142,0.968866,0.856086,-0.263644,0.904229,0.953434,0.577942
CHF,0.777986,0.984548,0.133959,0.940351,1.000000,0.800285,0.983222,0.984295,0.808153,0.942876,...,0.510947,0.985909,0.238119,0.796926,0.933201,0.755014,-0.118778,0.942786,0.955919,0.378904
CNY,0.795751,0.831253,0.237993,0.905083,0.800285,1.000000,0.826352,0.830521,0.838936,0.752308,...,0.088921,0.747899,-0.240729,0.892548,0.933197,0.958427,-0.513032,0.755497,0.856168,0.826735
DKK,0.833132,0.999586,0.219477,0.950745,0.983222,0.826352,1.000000,0.999740,0.814783,0.938879,...,0.535105,0.988112,0.212659,0.798258,0.950605,0.775312,-0.106199,0.938621,0.964455,0.425657
EUR,0.830476,0.999878,0.213423,0.951474,0.984295,0.830521,0.999740,1.000000,0.818601,0.941058,...,0.528736,0.988181,0.208518,0.800679,0.953292,0.781902,-0.111991,0.940912,0.966671,0.428241
GBP,0.704479,0.818435,-0.051492,0.884265,0.808153,0.838936,0.814783,0.818601,1.000000,0.783731,...,0.212605,0.783578,-0.071158,0.821335,0.871183,0.801667,-0.325053,0.785860,0.847732,0.571999


In [18]:
corr[corr != 1.0][corr > 0.98].stack()

kod  kod
BGN  CHF    0.984548
     DKK    0.999586
     EUR    0.999878
     HRK    0.985430
     RON    0.987958
CHF  BGN    0.984548
     DKK    0.983222
     EUR    0.984295
     RON    0.985909
DKK  BGN    0.999586
     CHF    0.983222
     EUR    0.999740
     HRK    0.985675
     RON    0.988112
EUR  BGN    0.999878
     CHF    0.984295
     DKK    0.999740
     HRK    0.985802
     RON    0.988181
HKD  USD    0.999949
HRK  BGN    0.985430
     DKK    0.985675
     EUR    0.985802
     RON    0.992426
MYR  SGD    0.988070
PHP  XDR    0.985301
RON  BGN    0.987958
     CHF    0.985909
     DKK    0.988112
     EUR    0.988181
     HRK    0.992426
SGD  MYR    0.988070
USD  HKD    0.999949
XDR  PHP    0.985301
dtype: float64

In [14]:
fig, ax = plt.subplots(figsize=(len(corr) / 3, len(corr) / 3))
cax = ax.matshow(corr, cmap='RdYlGn')
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90);
plt.yticks(range(len(corr.columns)), corr.columns);

# Add the colorbar legend
cbar = fig.colorbar(cax, ticks=[-1, 0, 1], aspect=40, shrink=.8)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Úkol C

Ve třetím úkolu, naším vlastním, jsme se rozhodli zjistit zda nemají jednotlivé dny v týdnu vliv na změnu kurzu.

In [24]:
cursor.execute(
    "SELECT date_part('dow', den::date) as dow, AVG(normalizovany_kurz) FROM kurz GROUP BY dow order by dow"
)
dow = ["sun", "mon", "tue", "wed", "thu", "fri", "sat"]
days = {dow[int(day)]: value for day, value in cursor}

fig = plt.figure()
x = np.arange(len(days))
plt.bar(x, height=days.values())
plt.xticks(x, days.keys(), rotation=-90);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
days

{'mon': 8.10166679292929,
 'tue': 8.1323463315508,
 'wed': 8.11074049431818,
 'thu': 8.12011061319074,
 'fri': 8.1168832789001}

Vidíme, že rozdíl mezi jednotlivými dny je téměř zanedbatelný, ač je znát mírný skok mezi hodnotami v pondělí a v úterý.

In [35]:
cursor.execute("""
SELECT date_part('dow', end_time::date) as dow, AVG(Change)
FROM (
  SELECT 
    LAG(den, 1) OVER (ORDER BY den) AS start_time,
    den AS end_time, 
    (normalizovany_kurz - LAG(normalizovany_kurz, 1) OVER (ORDER BY den)) AS Change
  FROM kurz
) res
GROUP BY dow
""")
dow = ["sun", "mon", "tue", "wed", "thu", "fri", "sat"]
days = {dow[int(day)]: value for day, value in cursor}

fig = plt.figure()
x = np.arange(len(days))
plt.bar(x, height=days.values())
plt.xticks(x, days.keys(), rotation=-90);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …